## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [ ]:
# Import libaries
import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import datetime

In [ ]:
# Load up store_income_data.csv
df = pd.read_csv('store_income_data_task.csv')

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [ ]:
# Print all the unique values in the 'country' column before cleaning
countries = df['country'].unique()
print(f"There are {len(countries)} unique countries before cleaning.\n")
print(countries)

# Convert the 'country' column to lowercase and remove any trailing white spaces
df['country'] = df['country'].str.lower().str.strip()

# Replace all country names that have null values with 'pending'
df.country.fillna('pending', inplace = True)

# Print all the unique values in the 'country' column after cleaning
countries = df['country'].unique()
print(f"\nThere are {len(countries)} unique countries after cleaning.\n")
print(countries)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [ ]:
# Determines the min_ratio when calling the replace_country_names function
matches = fuzzywuzzy.process.extract("s.a", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

In [ ]:
# Replaces country names that are a close match using fuzzy logic and a custom ratio
def replace_country_names(df, column, string_to_match, min_ratio):
    
    # Get a list of unique names for the specified column
    unique_strings = df[column].unique()

    # Get the top 10 closest matches for the string_to_match value
    matches = fuzzywuzzy.process.extract(string_to_match, unique_strings,
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio greater than or equal to min_ratio value
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match
    print("The replacement has been completed.")

replace_country_names(df=df, column='country', string_to_match="uk", min_ratio=40)
replace_country_names(df=df, column='country', string_to_match="united kingdom", min_ratio=90)
replace_country_names(df=df, column='country', string_to_match="britain", min_ratio=90)
replace_country_names(df=df, column='country', string_to_match="england", min_ratio=90)
replace_country_names(df=df, column='country', string_to_match="united states", min_ratio=70)
replace_country_names(df=df, column='country', string_to_match="america", min_ratio=90)
replace_country_names(df=df, column='country', string_to_match="south africa", min_ratio=75)
replace_country_names(df=df, column='country', string_to_match="s.a.", min_ratio=40)

In [ ]:
# Replaces the specified country name with a new name
def replace_name(string_to_find, new_name):
   
    df.replace(string_to_find, new_name, inplace=True)
    print("The country name has been replaced.")


replace_name(string_to_find='united states', new_name="United States")
replace_name(string_to_find='america', new_name="United States")
replace_name(string_to_find='s.a.', new_name="South Africa")
replace_name(string_to_find='britain', new_name="United Kingdom")
replace_name(string_to_find='united kingdom', new_name="United Kingdom")
replace_name(string_to_find='uk', new_name="United Kingdom")
replace_name(string_to_find='england', new_name="United Kingdom")
replace_name(string_to_find='/', new_name="pending")
replace_name(string_to_find='.', new_name="pending")
replace_name(string_to_find='', new_name="pending")

# Display unique country names after all cleaning
countries = df['country'].unique()
print(f"\nNumber of unique countries: {len(countries)}")
countries

In [ ]:
# Remove the 'pending' values in  countries to remain with  unique countries
filtered_df = df[df['country'] != 'pending']

# Print out the three distinct countries
filtered_df.country.unique()

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [ ]:
# Convert 'date_measured' from object type to datetime64 type
df['date_measured'] = pd.to_datetime(df['date_measured'], format='%d-%m-%Y')

# Copy 'days_ago' column from 'date_measured' column and convert to datetime64 type
df['days_ago'] = pd.to_datetime(df['date_measured'], format='%d-%m-%Y')
df['days_ago'] = pd.to_datetime(datetime.date.today(), format='%d-%m-%Y')

# Subtract the date measured from todays date to get the number of days that have passed
df['days_ago'] = df['days_ago'] - df['date_measured']
df.head()